In [1]:
pip install PyPDF2 anthropic openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter as tk
from tkinter import ttk, scrolledtext, filedialog, messagebox
import PyPDF2
from anthropic import Anthropic
import openai
import json
from datetime import datetime

# Configuração das APIs
OPENAI_API_KEY = ""
CLAUDE_API_KEY = ""

class PDFAnalyzerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Analisador de PDF - OpenAI vs Claude")
        self.root.geometry("1200x800")
        
             
        # Variáveis de controle
        self.pdf_content = ""
        self.selected_openai_model = tk.StringVar(value="gpt-4")
        self.selected_claude_model = tk.StringVar(value="claude-v1")

        # Inicializar clientes API
        self.setup_api_clients()

        # Interface principal
        self.setup_ui()
        
    def setup_api_clients(self):
        try:
            self.openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)
            self.anthropic = Anthropic(api_key=CLAUDE_API_KEY)
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao inicializar APIs: {str(e)}")
            raise SystemExit
        
    def setup_ui(self):
        # Frame para seleção e processamento do PDF
        input_frame = ttk.LabelFrame(self.root, text="Entrada", padding=10)
        input_frame.pack(fill="x", padx=10, pady=5)
        
        # Área para seleção do PDF
        pdf_frame = ttk.Frame(input_frame)
        pdf_frame.pack(fill="x", pady=5)
        
        self.pdf_path = ttk.Entry(pdf_frame, width=80)
        self.pdf_path.pack(side="left", padx=5)
        
        ttk.Button(pdf_frame, text="Selecionar PDF", command=self.select_pdf).pack(side="left", padx=5)
        ttk.Button(pdf_frame, text="Processar PDF", command=self.process_pdf).pack(side="left", padx=5)
        
        # Frame para prompts
        prompts_frame = ttk.Frame(input_frame)
        prompts_frame.pack(fill="x", pady=5)
        
        ttk.Label(prompts_frame, text="Prompt de Análise:").pack(anchor="w")
        self.analysis_prompt = scrolledtext.ScrolledText(prompts_frame, height=3)
        self.analysis_prompt.pack(fill="x", pady=5)
        self.analysis_prompt.insert("1.0", """Elabore um relatório de sentença de mérito trabalhista, estruturado em formato narrativo e com redação objetiva, sempre utilizando as abreviações "rte." e "rda." sem mencionar o nome das partes. No relatório da rda., utilize sempre a construção "que o rte. não trabalhou em sobrejornada; que o rte. não faz jus a ...". Comece sempre com "O(a) rte. pede ...tal tal tal". Não cite pedidos referentes a honorários e justiça gratuita em geral. A fundamentação deve incluir o título do tópico "Das horas extras". Siga o seguinte padrão: inicie apresentando as alegações do autor sobre os fatos e pedidos formulados, começando com "O(a) rte. pede...", em seguida exponha de forma contínua as impugnações e argumentos apresentados pela defesa da reclamada, utilizando as abreviações "rte." e "rda." sem mencionar o nome das partes. Não analise o mérito dos pedidos nem julgue o processo. O texto não deve utilizar listas, tópicos ou subdivisões. Deve ser fluido, técnico e redigido em terceira pessoa, com linguagem clara e direta, mantendo o tom impessoal e a formalidade típica de decisões judiciais trabalhistas. Evite qualquer tipo de análise ou opinião, limitando-se à descrição dos fatos e alegações conforme os autos.""")
        
        ttk.Label(prompts_frame, text="Prompt de Comparação:").pack(anchor="w")
        self.comparison_prompt = scrolledtext.ScrolledText(prompts_frame, height=3)
        self.comparison_prompt.pack(fill="x", pady=5)
        self.comparison_prompt.insert("1.0", "Você é um juiz de direito especializado em análise de processos trabalhistas. Sua tarefa é analisar e comparar dois resumos gerados por diferentes sistemas de inteligência artificial sobre um processo trabalhista e compará-los ao processo principal. O objetivo é garantir que todas as informações importantes estejam presentes, identificar lacunas e avaliar qual resumo é mais completo e fiel ao processo. Você receberá dois resumos do processo trabalhista, além de informações do processo principal, quando fornecidas. Sua análise deve seguir os seguintes passos: Primeiro, compare os dois resumos, identificando as informações que aparecem em ambos, as que aparecem apenas no primeiro resumo e as que aparecem apenas no segundo. Em seguida, compare os resumos com o processo principal para verificar se todas as informações importantes do processo estão presentes em pelo menos um dos resumos e se há informações nos resumos que não constam no processo principal. Identifique quaisquer fatos essenciais que estejam ausentes em ambos os resumos. Por fim, forneça uma avaliação geral da qualidade e completude dos resumos em relação ao processo principal, indicando qual dos resumos é mais completo e fiel. Certifique-se de ser claro, objetivo e detalhado em sua análise, focando nas informações mais relevantes do processo trabalhista e destacando qualquer falha significativa ou omissão.")

        ttk.Label(prompts_frame, text="Prompt de Análise Híbrida:").pack(anchor="w")
        self.hybrid_prompt = scrolledtext.ScrolledText(prompts_frame, height=3)
        self.hybrid_prompt.pack(fill="x", pady=5)
        self.hybrid_prompt.insert("1.0", "Una dois resumos e crie um texto único com as informações deles, sem redundância, em um texto coeso, conciso e formal, estruturado com parágrafos bem definidos, linguagem técnica e impessoal, sem listas ou bullets, como em decisões judiciais trabalhistas, com clareza e precisão. Sempre utilize as abreviações rte. e rda., omitindo o nome da parte. No relatório da rda., empregue sempre a construção 'que o rte. não trabalhou em sobrejornada; que o rte. não faz jus a ...'. Inicie sempre com 'O(a) rte. pede ... tal tal tal'. Não mencione pedidos referentes a honorários e justiça gratuita em geral. Na fundamentação, inclua o título dos tópicos, por exemplo, 'Das horas extras'.")

        ttk.Label(prompts_frame, text="Prompt Customizado:").pack(anchor="w")
        self.custom_prompt = scrolledtext.ScrolledText(prompts_frame, height=3)
        self.custom_prompt.pack(fill="x", pady=5)
        self.custom_prompt.insert("1.0", "Insira seu prompt personalizado aqui...")

        # Frame para seleção de modelos
        model_frame = ttk.LabelFrame(input_frame, text="OpenAI", padding=10)
        model_frame.pack(side="left", padx=5, pady=5)
        openai_models = ["gpt-4o", "gpt-4o-mini", "o1-mini"]
        self.create_model_selection(model_frame, "Modelo OpenAI:", openai_models, self.selected_openai_model)
        model_frame = ttk.LabelFrame(input_frame, text="Claude", padding=10)
        model_frame.pack(side="left", padx=5, pady=5)
        claude_models = ["claude-3-5-sonnet-latest", "claude-3-5-haiku-latest", "claude-3-opus-latest"]
        self.create_model_selection(model_frame, "Modelo Claude:", claude_models, self.selected_claude_model)

        # Frame para ações
        actions_frame = ttk.Frame(self.root)
        actions_frame.pack(fill="x", padx=10, pady=5)
        
        ttk.Button(actions_frame, text="Comparar Resultados", command=self.compare_results).pack(side="left", padx=5)
        ttk.Button(actions_frame, text="Salvar Resultados", command=self.save_results).pack(side="left", padx=5)
        ttk.Button(actions_frame, text="Gerar Resumo Híbrido", command=self.generate_hybrid).pack(side="left", padx=5)
        ttk.Button(actions_frame, text="Processar Prompt Customizado", command=self.process_custom_prompt).pack(side="left", padx=5)
        
       # Barra de progresso
        self.progress_var = tk.DoubleVar()
        self.progress_bar = ttk.Progressbar(self.root, variable=self.progress_var, maximum=100)
        self.progress_bar.pack(fill="x", padx=10, pady=5)

        # Notebook para resultados
        results_frame = ttk.LabelFrame(self.root, text="Resultados", padding=10)
        results_frame.pack(fill="both", expand=True, padx=10, pady=5)

        self.notebook = ttk.Notebook(results_frame)
        self.notebook.pack(fill="both", expand=True)

        # Abas para resultados
        openai_tab = ttk.Frame(self.notebook)
        claude_tab = ttk.Frame(self.notebook)
        comparison_tab = ttk.Frame(self.notebook)
        hybrid_tab = ttk.Frame(self.notebook)
        custom_tab = ttk.Frame(self.notebook)
        
        self.notebook.add(openai_tab, text="OpenAI")
        self.notebook.add(claude_tab, text="Claude")
        self.notebook.add(comparison_tab, text="Comparação")
        self.notebook.add(hybrid_tab, text="Resumo Híbrido")
        self.notebook.add(custom_tab, text="Análise Customizada")
        
        self.openai_result = scrolledtext.ScrolledText(openai_tab)
        self.claude_result = scrolledtext.ScrolledText(claude_tab)
        self.comparison_result = scrolledtext.ScrolledText(comparison_tab)
        self.hybrid_result = scrolledtext.ScrolledText(hybrid_tab)
        self.custom_result = scrolledtext.ScrolledText(custom_tab)
        
        self.openai_result.pack(fill="both", expand=True)
        self.claude_result.pack(fill="both", expand=True)
        self.comparison_result.pack(fill="both", expand=True)
        self.hybrid_result.pack(fill="both", expand=True)
        self.custom_result.pack(fill="both", expand=True)
        
        
    def create_model_selection(self, frame, label_text, options, var):
        ttk.Label(frame, text=label_text).pack(anchor="w")
        for option in options:
            ttk.Radiobutton(frame, text=option, value=option, variable=var).pack(anchor="w")
        
    def select_pdf(self):
        file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if file_path:
            self.pdf_path.delete(0, tk.END)
            self.pdf_path.insert(0, file_path)
    
    def process_pdf(self):
        if not self.pdf_path.get():
            messagebox.showerror("Erro", "Selecione um PDF primeiro!")
            return
            
        try:
            # Extrair texto do PDF
            self.progress_var.set(10)
            self.root.update_idletasks()
            
            with open(self.pdf_path.get(), 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                self.pdf_content = ""
                total_pages = len(reader.pages)
                
                for i, page in enumerate(reader.pages):
                    self.pdf_content += page.extract_text()
                    self.progress_var.set(10 + (i + 1) / total_pages * 30)
                    self.root.update_idletasks()
            
            if not self.pdf_content.strip():
                raise Exception("PDF vazio ou ilegível")
            
            # Análise com OpenAI
            self.progress_var.set(40)
            self.root.update_idletasks()
            
            analysis_prompt = self.analysis_prompt.get("1.0", tk.END).strip()
            openai_response = self.openai_client.chat.completions.create(
                model=self.selected_openai_model.get(),
                messages=[
                    {"role": "system", "content": "Você é um assistente especializado em análise de documentos."},
                    {"role": "user", "content": f"{analysis_prompt}\n\nDocumento:\n{self.pdf_content}"}
                ]
            )
            
            self.openai_result.delete("1.0", tk.END)
            self.openai_result.insert("1.0", openai_response.choices[0].message.content)
            
            # Análise com Claude
            self.progress_var.set(70)
            self.root.update_idletasks()
            
            
            claude_response = self.anthropic.messages.create(
                model=self.selected_claude_model.get(),
                max_tokens=4096,
                messages=[
                    {"role": "user", "content": f"{analysis_prompt}\n\nDocumento:\n{self.pdf_content}"}
                ]
            )
            
            self.claude_result.delete("1.0", tk.END)
            self.claude_result.insert("1.0", claude_response.content[0].text)
            
            self.progress_var.set(100)
            messagebox.showinfo("Sucesso", "PDF processado com sucesso!")
            
        except Exception as e:
            self.progress_var.set(0)
            messagebox.showerror("Erro", f"Erro ao processar PDF: {str(e)}")

    def generate_hybrid(self):
        if not self.openai_result.get("1.0", tk.END).strip() or not self.claude_result.get("1.0", tk.END).strip():
            messagebox.showerror("Erro", "Processe o PDF e gere os resumos primeiro!")
            return
        
        try:
            hybrid_prompt = self.hybrid_prompt.get("1.0", tk.END).strip()
            openai_text = self.openai_result.get("1.0", tk.END).strip()
            claude_text = self.claude_result.get("1.0", tk.END).strip()
            
            prompt_with_summaries = f"""
            Análise Comparativa de Resumos Processuais

            RESUMO OPENAI:
            {openai_text}

            RESUMO CLAUDE:
            {claude_text}

            DIRETRIZES PARA ANÁLISE SINTÉTICA:
            {hybrid_prompt}
            """
            
            hybrid_response = self.openai_client.chat.completions.create(
                model=self.selected_openai_model.get(),
                messages=[
                    {"role": "system", "content": "Você é um assistente especializado em análise de documentos."},
                    {"role": "user", "content": prompt_with_summaries}
                ]
            )
            
            self.hybrid_result.delete("1.0", tk.END)
            self.hybrid_result.insert("1.0", hybrid_response.choices[0].message.content)
            
            self.progress_var.set(100)
            self.notebook.select(3)
            messagebox.showinfo("Sucesso", "Resumo híbrido gerado com sucesso!")
            
        except Exception as e:
            self.progress_var.set(0)
            messagebox.showerror("Erro", f"Erro ao gerar resumo híbrido: {str(e)}")
    
    def compare_results(self):
        if not self.openai_result.get("1.0", tk.END).strip() or not self.claude_result.get("1.0", tk.END).strip():
            messagebox.showerror("Erro", "Processe o PDF primeiro!")
            return
        
        try:
            self.progress_var.set(50)
            comparison_prompt = self.comparison_prompt.get("1.0", tk.END).strip()
            openai_text = self.openai_result.get("1.0", tk.END).strip()
            claude_text = self.claude_result.get("1.0", tk.END).strip()
            
            comparison_response = self.anthropic.messages.create(
                model=self.selected_claude_model.get(),
                max_tokens=4096,
                messages=[
                    {"role": "user", "content": f"{comparison_prompt}\n\nDocumento Original:\n{self.pdf_content}\n\nResultado OpenAI:\n{openai_text}\n\nResultado Claude:\n{claude_text}"}
                ]
            )
            
            self.comparison_result.delete("1.0", tk.END)
            self.comparison_result.insert("1.0", comparison_response.content[0].text)
            
            self.progress_var.set(100)
            self.notebook.select(2)
            messagebox.showinfo("Sucesso", "Comparação concluída!")
            
        except Exception as e:
            self.progress_var.set(0)
            messagebox.showerror("Erro", f"Erro na comparação: {str(e)}")

    def process_custom_prompt(self):
        try:
            self.progress_var.set(30)
            custom_prompt = self.custom_prompt.get("1.0", tk.END).strip()
        
            if not custom_prompt or custom_prompt == "Insira seu prompt personalizado aqui...":
                messagebox.showerror("Erro", "Insira um prompt válido!")
                return
            
                self.progress_var.set(50)
                custom_response = self.openai_client.chat.completions.create(
                    model=self.selected_openai_model.get(),
                    messages=[
                        {"role": "system", "content": "Você é um assistente especializado em análise de documentos."},
                        {"role": "user", "content": f"{custom_prompt}\n\nDocumento:\n{self.pdf_content}"}
                    ]
                )
                
            self.custom_result.delete("1.0", tk.END)
            self.custom_result.insert("1.0", custom_response.choices[0].message.content)
                
            self.progress_var.set(100)
            self.notebook.select(4)
            messagebox.showinfo("Sucesso", "Análise customizada concluída!")
            
        except Exception as e:
                self.progress_var.set(0)
                messagebox.showerror("Erro", f"Erro ao processar prompt customizado: {str(e)}")
            

    def save_results(self):
        if not self.comparison_result.get("1.0", tk.END).strip():
            messagebox.showerror("Erro", "Não há resultados para salvar!")
            return
        
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            results = {
                "pdf_path": self.pdf_path.get(),
                "analysis_prompt": self.analysis_prompt.get("1.0", tk.END).strip(),
                "comparison_prompt": self.comparison_prompt.get("1.0", tk.END).strip(),
                "openai_result": self.openai_result.get("1.0", tk.END).strip(),
                "claude_result": self.claude_result.get("1.0", tk.END).strip(),
                "comparison_result": self.comparison_result.get("1.0", tk.END).strip(),
                "hybrid_result": self.hybrid_result.get("1.0", tk.END).strip(),
                "custom_prompt": self.custom_prompt.get("1.0", tk.END).strip(),
                "custom_result": self.custom_result.get("1.0", tk.END).strip(),
                "timestamp": timestamp
            }
            
            file_path = filedialog.asksaveasfilename(
                defaultextension=".json",
                filetypes=[("JSON files", "*.json")],
                initialfile=f"analise_pdf_{timestamp}.json"
            )
            
            if file_path:
                with open(file_path, 'w', encoding='utf-8') as f:
                    json.dump(results, f, ensure_ascii=False, indent=2)
                messagebox.showinfo("Sucesso", "Resultados salvos com sucesso!")
                
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao salvar resultados: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = PDFAnalyzerApp(root)
    root.mainloop()